In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import  Dense, Flatten, Dropout

In [2]:
#importing the necessary libraries
import tensorflow as tf
import gc
from tensorflow import keras
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from datetime import datetime 
from sklearn.metrics import roc_auc_score

In [3]:
import matplotlib.pyplot as plt
import imblearn
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
#loading the dataset
file = tf.keras.utils
df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')

In [5]:
#checking the dataset
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [6]:
cleaned_df = df.copy()

# You don't want the `Time` column.
cleaned_df.pop('Time')

# The `Amount` column covers a huge range. Convert to log-space.
eps = 0.001 # 0 => 0.1¢
cleaned_df['Log Amount'] = np.log(cleaned_df.pop('Amount')+eps)

In [7]:
#time
df['Hour'] = df['Time'].apply(lambda x: np.floor(x / 3600))

tmp = df.groupby(['Hour', 'Class'])['Amount'].aggregate(['min', 'max', 'count', 'sum', 'mean', 'median', 'var']).reset_index()
df_2 = pd.DataFrame(tmp)
df_2.columns = ['Hour', 'Class', 'Min', 'Max', 'Transactions', 'Sum', 'Mean', 'Median', 'Var']
df_2.head()

,Hour,Class,Min,Max,Transactions,Sum,Mean,Median,Var
0,0.0,0,0.0,7712.43,3961,256572.87,64.774772,12.990,45615.821201
1,0.0,1,0.0,529.00,2,529.00,264.500000,264.500,139920.500000
2,1.0,0,0.0,1769.69,2215,145806.76,65.826980,22.820,20053.615770
3,1.0,1,59.0,239.93,2,298.93,149.465000,149.465,16367.832450
4,2.0,0,0.0,4002.88,1555,106989.39,68.803466,17.900,45355.430437


In [8]:
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [10]:
#Normalize the input features using the sklearn StandardScaler. 
#This will set the mean to 0 and standard deviation to 1.
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

Training labels shape: (182276,)
Validation labels shape: (45569,)
Test labels shape: (56962,)
Training features shape: (182276, 29)
Validation features shape: (45569, 29)
Test features shape: (56962, 29)


In [11]:
#oversampling on training data
print('Original dataset shape %s' % Counter(train_labels))

ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(train_df,train_labels)

print('Resampled dataset shape %s' % Counter(y_train_ros))

Original dataset shape Counter({0: 181976, 1: 300})
Resampled dataset shape Counter({0: 181976, 1: 181976})


In [12]:
#oversampling on testing data
print('Original dataset shape %s' % Counter(test_labels))

ros = RandomOverSampler(random_state=42)
X_test_ros, y_test_ros = ros.fit_resample(test_df,test_labels)

print('Resampled dataset shape %s' % Counter(y_test_ros))

Original dataset shape Counter({0: 56855, 1: 107})
Resampled dataset shape Counter({0: 56855, 1: 56855})


In [13]:
#oversampling on testing data
print('Original dataset shape %s' % Counter(val_labels))

ros = RandomOverSampler(random_state=42)
X_val_ros, y_val_ros = ros.fit_resample(val_df, val_labels)

print('Resampled dataset shape %s' % Counter(y_val_ros))

Original dataset shape Counter({0: 45484, 1: 85})
Resampled dataset shape Counter({0: 45484, 1: 45484})


In [14]:
#training outputs
Y_train_ros = np.zeros([len(y_train_ros),2])
for y in range(len(y_train_ros)):
    if y_train_ros[y]==1:
        Y_train_ros[y,1] = 1
    else:
        Y_train_ros[y,0] = 1  

In [15]:
#preparing the data for training
Y_val_ros = np.zeros([len(y_val_ros),2])

for y in range(len(y_val_ros)):
    if y_val_ros[y]==1:
        Y_val_ros[y,1] = 1
    else:
        Y_val_ros[y,0] = 1  

In [16]:
#preparing data for validation
Y_test_ros = np.zeros([len(y_test_ros),2])
for y in range(len(y_test_ros)):
    if y_test_ros[y]==1:
        Y_test_ros[y,1] = 1
    else:
        Y_test_ros[y,0] = 1 

In [17]:
# Creation of the model layers
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_ros.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='sigmoid')
])


In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [19]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
epochs = 25
history = model.fit(X_train_ros, Y_train_ros, batch_size=32, epochs=epochs, validation_data=(X_val_ros,Y_val_ros))
model.save("Fraud transaction detection")

Epoch 1/25
11374/11374 [==============================] - 33s 3ms/step - loss: 0.0675 - accuracy: 0.9805 - val_loss: 0.5103 - val_accuracy: 0.9271
Epoch 2/25
11374/11374 [==============================] - 35s 3ms/step - loss: 0.0185 - accuracy: 0.9965 - val_loss: 0.7853 - val_accuracy: 0.9049
Epoch 3/25
11374/11374 [==============================] - 39s 3ms/step - loss: 0.0131 - accuracy: 0.9977 - val_loss: 0.7860 - val_accuracy: 0.9109
Epoch 4/25
11374/11374 [==============================] - 41s 4ms/step - loss: 0.0102 - accuracy: 0.9984 - val_loss: 0.9223 - val_accuracy: 0.9048
Epoch 5/25
11374/11374 [==============================] - 43s 4ms/step - loss: 0.0089 - accuracy: 0.9986 - val_loss: 1.0619 - val_accuracy: 0.9053
Epoch 6/25
11374/11374 [==============================] - 40s 3ms/step - loss: 0.0073 - accuracy: 0.9989 - val_loss: 1.0178 - val_accuracy: 0.9054
Epoch 7/25
11374/11374 [==============================] - 34s 3ms/step - loss: 0.0062 - accuracy: 0.9990 - val_loss: 1

In [21]:
#predicting testing data
preds = model.predict(X_test_ros)

3554/3554 [==============================] - 5s 1ms/step


In [22]:
#identifying the class
preds = np.argmax(preds, axis=1)

In [23]:
#accuracy of the testing data
accuracy = accuracy_score(y_test_ros, preds)

In [24]:
accuracy

0.9151613754287222

In [25]:
# Calculate the classification report
report = classification_report(y_test_ros, preds)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92     56855
           1       1.00      0.83      0.91     56855

    accuracy                           0.92    113710
   macro avg       0.93      0.92      0.91    113710
weighted avg       0.93      0.92      0.91    113710

